In [8]:
import pandas as pd
import numpy as np
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum
from optimization import MaintenanceOptimization

In [26]:
def sensitivity_analysis(optimization_model, parameter_name, parameter_values):
    """
    Perform sensitivity analysis by varying a given parameter and re-solving the optimization problem.

    Args:
        optimization_model (MaintenanceOptimization): The initialized optimization model.
        parameter_name (str): The parameter to vary (e.g., 'downtime_cost_per_kwh').
        parameter_values (list): List of values to test for the parameter.

    Returns:
        pd.DataFrame: Results of the sensitivity analysis including parameter values, objective values, and adjusted downtime costs.
    """
    results = []

    for value in parameter_values:
        # Update the parameter being analyzed
        if parameter_name == 'downtime_cost_per_kwh':
            optimization_model.downtime_cost_per_kwh = value
        elif parameter_name in optimization_model.maintenance_costs:
            optimization_model.maintenance_costs[parameter_name] = value

        simulated_downtime = {f"Downtime_{i}": 12 for i in optimization_model.data.index}  # Half of 24 hours

        # Calculate hypothetical downtime costs
        simulated_downtime_cost = sum(
            optimization_model.data.loc[i, 'WEC: Production kWh'] * simulated_downtime[f"Downtime_{i}"] * value
            for i in optimization_model.data.index
        )

        # Add simulated downtime cost to the fixed maintenance cost
        fixed_maintenance_cost = sum(optimization_model.maintenance_costs['routine'] if optimization_model.data.loc[i, 'Fault'] == 'Routine' else optimization_model.maintenance_costs['default']
                                     for i in optimization_model.data.index)
        
        adjusted_total_cost = fixed_maintenance_cost + simulated_downtime_cost

        # Append results
        results.append({
            'Parameter': parameter_name,
            'Value': value,
            'Simulated_Downtime_Cost': simulated_downtime_cost,
            'Adjusted_Total_Cost': adjusted_total_cost,
            'Simulated_Downtime': simulated_downtime,
        })

    return pd.DataFrame(results)

In [27]:
# Perform sensitivity analysis on downtime cost per kWh
parameter_name = 'downtime_cost_per_kwh'
parameter_values = [0.05, 0.06, 0.07, 0.08, 0.09]  # Test different values of downtime cost

# Run sensitivity analysis
results = sensitivity_analysis(model, parameter_name, parameter_values)

# Display results
print(results[['Parameter', 'Value', 'Simulated_Downtime_Cost', 'Adjusted_Total_Cost']])

               Parameter  Value  Simulated_Downtime_Cost  Adjusted_Total_Cost
0  downtime_cost_per_kwh   0.05                   3780.0            1603780.0
1  downtime_cost_per_kwh   0.06                   4536.0            1604536.0
2  downtime_cost_per_kwh   0.07                   5292.0            1605292.0
3  downtime_cost_per_kwh   0.08                   6048.0            1606048.0
4  downtime_cost_per_kwh   0.09                   6804.0            1606804.0
